### Modelling the grammar of the Psalms

This notebook will explore the use of grammatical features to classify sentences from the book of Genesis and the Psalms. The basic idea is to build a dataset of observations with each row representing a sentence and each column representing the count of a grammatical feature within the sentence. This dataset will then be used with a logistic regression model to investigate which grammatical features are most significant in determining whether a sentence is more likely to be from Genesis or Psalms. (I chose logistic regression because it is simple and interpretable, in the future it will be a good idea to try other models). 

The goal of the notebook is to illustrate a technique for incorporating statistical modelling into the analysis of biblical literature. Note that I have not put a great deal of effort into refining the model at this point and it could surely be improved by thinking a little more carefully about which features to capture and record. 

In [171]:
# Imports
from nltk import *
from tf.app import use
import random
A = use('bhsa', hoist=globals())

#### Tree Functions

The following functions are used to convert the BHSA data into NLTK tree structure. The linguistic structure can then be more easily analyzed from the trees.

In [172]:
#------------------------------------------------------------------------------------------
        
def get_pos(node, tree):
    ''' given the number of a node and a tree, returns the pos of that node in the tree '''
    for pos in tree.treepositions():
        try:
            tree[pos].label()
        except AttributeError:
            pass
        else:
            #pos_node = int(tree[pos].label()[tree[pos].label().index(':')+1:])
            pos_node = int(get_node(pos, tree))
            if node == pos_node:
                node_pos = pos
                
    return node_pos

#------------------------------------------------------------------------------------------

def get_node(pos, tree):
    '''given a position in a tree, returns the number of the node at that position'''
    try:
        node = tree[pos].label()[tree[pos].label().index(':')+1:]
    except:
        node = None
        
    return node

#------------------------------------------------------------------------------------------

def get_label(pos,tree):
    '''given a position in a tree, returns the label of the tree at that position'''
    try:
        label = tree[pos].label()[:tree[pos].label().index(':')]
    except:
        label = tree[pos].label()
        
    return label

#------------------------------------------------------------------------------------------

def tree_string(node):
    
    ''' Takes BHSA node as root and adds all lower distributional objects to a tree '''
    
    slots = []
    slotlist = []
    for s in E.oslots.s(node):
        slotlist.append(s)
    slots.append((node, slotlist))
    
    for n in L.d(node):
        if F.otype.v(n) == 'word':
            slots.append((n, [E.oslots.s(n)[0]]))
        else:
            slotlist = []
            for s in E.oslots.s(n):
                slotlist.append(s)
            slots.append((n, slotlist))
    parse_string = []
    root_node = '(S:' + str(node)
    parse_string.append(root_node)

    for slot in slots[0][1]:
        parse_string.append(slot)
    
    for (label, slot) in slots:
    
        l = ''
        add = False
        if F.otype.v(label) == 'sentence_atom':
            l = 'S_A'
            add = True
        elif F.otype.v(label) == 'clause_atom':
            l = 'C_A'
            add = True
        elif F.otype.v(label) == 'phrase_atom':
            l = 'P_A'
            add = True
        elif F.otype.v(label) == 'subphrase':
            func = F.function.v(label)
            if func == None:
                l = 'U'
            else:
                l = func
            add = True

        elif F.otype.v(label) == 'word':
            l = F.sp.v(label)
            add = True
        
        l = l + ":" + str(label)
    
        if add:
            open_index = parse_string.index(slot[0])
            parse_string.insert(open_index,'('+l)
            close_index = parse_string.index(slot[-1])
            parse_string.insert(close_index + 1,')')

    parse_string.append(')')
    new_string = []
    for n in parse_string:
        if type(n) == int:
            #new_string.append(F.g_cons.v(n))
            new_string.append(str(n))
        else:
            new_string.append(n)
        
    return(' '.join(new_string))

#------------------------------------------------------------------------------------------

def move_func(t):
        
    ''' Takes distributional tree and rearranges under functional nodes '''
    levels = [('phrase', 'P'), ('clause','C')]
    for level in levels:
        atoms = []
        for pos in t.treepositions():
            try:
                t[pos].label()
            except AttributeError:
                pass
            else:
                label = t[pos].label()[:t[pos].label().index(':')]
                if label == level[1] +'_A':
                    atoms.append((pos, label, t[pos].leaves()))

        units = []
        labels = []
        for n in L.d(node):
            if F.otype.v(n) == level[0]:
                label = level[1] + ':' + str(n)
                labels.append(label)
                units.append((label, E.oslots.s(n)))
        units_table = {k:[] for k in labels}

        # for each unit find all atoms contained in that unit and append tree pos to dict
        for unit in units:
            for atom in atoms:
                result =  all(int(elem) in unit[1] for elem in atom[2])
                if result:
                    units_table[unit[0]].append(atom[0])

        # iterate through labels and build tree from atoms 
        new_trees = []
        deletes = []
        for label in labels:
            unit_atoms = units_table[label]
            trees = []
            i = unit_atoms[0][:-1]
            for atom in unit_atoms:
                tree = t[atom]
                trees.append(tree)
            new_tree = Tree(label, trees)
            new_trees.append((i, new_tree))
            for atom in unit_atoms:
                deletes.append(atom)
                
        # delete subtrees that are going to move
        for pos in reversed(t.treepositions()):
            if pos in deletes:
                del t[pos]

        # add subtrees in new spot
        for tree in reversed(new_trees):
            t[tree[0]].insert(0, tree[1])

#------------------------------------------------------------------------------------------

def move_daughter(t, relations):
    
    ''' Given a functional tree, rearranges dependent phrases into a hierarchical structure. '''

    phrases = []
    
    # traverse tree to find all nodes that may have dependent relations
    for pos in reversed(t.treepositions()):
        try:
            t[pos].label()
        except AttributeError:
            pass
        else:
            label = get_label(pos, t)
            if label == 'U' or label == 'P_A' or label == 'C':
                node = int(get_node(pos, t))
                rela = F.rela.v(node)
                if rela in relations:
                    mother = E.mother.f(node)[0]
                    phrases.append((node, mother, rela))

    for phrase in phrases:
        m_node = phrase[1]
        m_pos = get_pos(m_node, t)
        d_node = phrase[0]
        d_pos = get_pos(d_node, t)
        copy_tree = t[d_pos]
        
        mother_pos = t[m_pos].leaves()
        daughter_pos = copy_tree.leaves()

        if F.otype.v(m_node) == 'word' or phrase[2] == 'Link':
            # in these cases insert daughter into node *above* mother
            # insert point will be based on position of mother node in this node
            i_tree = m_pos[:-1]
            if mother_pos[0] < daughter_pos[0]:
                i_point = m_pos[-1:][0] + 1
            else:
                i_point = m_pos[-1:][0] - 1
        else:
            # otherwise insert point is position of daughter relative to mother
            i_tree = m_pos
            if mother_pos[0] < daughter_pos[0]:
                try:
                    i_point = mother_pos.index(str(int(daughter_pos[0])-1)) + 1
                except:
                    i_point = len(mother_pos)
            else:
                i_point = 0
        
        ## move tree by inserting at new position and then deleting at old position
        t[i_tree].insert(i_point, copy_tree)
        del t[d_pos]

#------------------------------------------------------------------------------------------

def move_para(t):
    
    ''' Given a tree, rearranges parrallel phrases under a new common node. '''
    relations = ['Para', 'Coor']
    phrases = []
    
    # find phrases with relation 'Para' or 'Coor'
    for pos in t.treepositions():
        try:
            t[pos].label()
        except AttributeError:
            pass
        else:
            label = get_label(pos, t)
            if label == 'P' or label == 'P_A' or label == 'C':
                node = int(get_node(pos, t))
                rela = F.rela.v(node)
                if rela in relations:
                    mother = E.mother.f(node)[0]
                    phrases.append((node, mother, rela))

    
    mothers = []
    daughters = []

    for phrase in phrases:
        mother = phrase[1]
        daughter = phrase[0]
        mothers.append(mother)
        daughters.append((mother, daughter))
    
    # build a dict since there may be more than one daughter parallel to the same mother
    table = {k:[] for k in mothers}
    for d in daughters:
        table[d[0]].append(d[1])
        
    ## get pos for mother and all daughters then build new tree under common node
    for mother in reversed(table):
        deletes = []
        m_pos = get_pos(mother, t)
        para_trees = []
        para_trees.append(t[m_pos])
        for daughter in table[mother]:
            d_pos = get_pos(daughter, t)
            para_trees.append(t[d_pos])
            deletes.append(d_pos)
            #del t[d_pos] # delete daughter at current position
        old_label = get_label(m_pos, t)
        new_label = old_label + ':000'
        new_tree = Tree(new_label, para_trees)
        
        # delete mother at old position
        del t[m_pos]

        # insert new para phrase in mother position
        i_tree = m_pos[:-1]
        i_point = m_pos[-1:][0]
        t[i_tree].insert(i_point, new_tree)
        
        # delete daughters
        for pos in reversed(t.treepositions()):
            if pos in deletes:
                del t[pos]
#------------------------------------------------------------------------------------------

def trim_nodes(t):
    ''' This function finds empty nodes left after rearrangement and trims them off. '''
    deletes = []
    for pos in t.treepositions():
        if len(t[pos]) == 0:
            deletes.append(pos)
    
    for delete in reversed(deletes):
        del t[delete]

#------------------------------------------------------------------------------------------

def collapse_nodes(t):
    ''' This function looks for redundant nodes and collapses them. Redundant nodes include 
    remaining sentence atoms and clause atoms, or other cases where two nodes of the same class 
    containing the same items are in a mother-daughter relationship   '''
    adds = []
    for pos in reversed(t.treepositions()[1:]):
        try:
            t[pos].label()
        except AttributeError:
            pass
        else:
            match = all(elem in t[pos].leaves() for elem in t[pos[:-1]].leaves())
            if ((match and ((t[pos].label()[0] == t[pos[:-1]].label()[0]) or (t[pos].label()[0] == 'U' and t[pos[:-1]].label()[0] == 'P'))) 
                or (t[pos].label().startswith('C_A'))
                or (t[pos].label().startswith('S_A'))):
                children = t[pos][0:]
                i_point = pos[-1:][0]
                del t[pos]
                for child in reversed(children):
                    t[pos[:-1]].insert(i_point, child)

#------------------------------------------------------------------------------------------

def relabel_tree(t):
    ''' This function changes tree labels to reflect phrase/clause type and function as necessary'''
    
    # these dicts are for converting labels to my preferred (often simpler) form, but this part can use some work 
    label_dict = {'adjv': 'adj',
                  'adjective': 'adj',
                  'advb': 'adv',
                  'adverb': 'adv',
                  'art': 'def',
                  'article': 'def',
                  'conj': 'conj',
                  'conjunction': 'conj',
                  'inrg': 'ir',
                  'interrogative': 'ir',
                  'intj': 'ij',
                  'interjection': 'ij',
                  'nega': 'neg',
                  'negative': 'neg',
                  'nmpr': 'pn',
                  'pronoun': 'pro',
                  'prde': 'pro-dem',
                  'prep': 'p',
                  'preposition': 'p',
                  'prin': 'pro-int',
                  'prps': 'pro-ps',
                  'subs': 'n',
                  'noun': 'n',
                  'verb': 'v',
                 }
    np = ['NP', 'PrNP', 'PPrP', 'DPrP', 'IPrP'] 
    
    
    sub_dict = {'adv':'AdvP',
                'n':'NP',
                'pn':'NP',
                'def':'NP',  # need to double check this
                'adj':'AdjP',
                'p':'PP',
                'v':'VP' # this is going to hit on some infinitives/participles, not sure if really VP
               }
    
    # traverse the tree and update each label
    for pos in reversed(t.treepositions()):
        try:
            t[pos].label()
        except AttributeError:
            pass
        else:
            label = get_label(pos, t)
            try:
                t[pos].set_label(label_dict[label])
            except:  
                node = int(get_node(pos,t))
                node_type = F.otype.v(node)
                func = F.function.v(node)
                rela = F.rela.v(node)
                typ = F.typ.v(node)
                if typ in np:
                    typ = 'NP'
                
                # 1. In case of Coord or Para I added new common node 0, pull lable from first child
                if node == 0:
                    new_label = t[pos][0].label()
                elif node_type == 'sentence' or node_type == 'sentence_atom':
                    new_label = 'SENT'
                elif node_type == 'clause' or node_type == 'clause_atom':
                    if rela == 'NA':
                        if typ == None:
                            new_label = 'CLAUSE'
                        else:
                            new_label = 'CLAUSE_' + typ
                    else:
                        new_label = 'CLAUSE_' + rela
                else:
                    if typ == 'NegP':
                        typ = 'NEG'
                        func = ''
                    elif typ == 'InjP':
                        typ = 'INJ'
                        func = ''
                    elif typ == 'CP':
                        if func == 'Rela':
                            typ = 'REL'
                            func = ''
                        else:
                            typ = 'CONJ'
                            func = ''
                    elif func == None:
                        if rela == 'NA' or rela == None:
                            func = ''
                        else:
                            func = '_' + rela
                    else:
                        func = '_' + func
                    
                    if typ == None:
                        ## need function to get type of subphrase
                        if node_type == 'subphrase':
                            try:
                                lower = t[pos][0].label()
                                if lower == 'def':
                                    lower = t[pos][1].label()
                                typ = sub_dict[lower]
                            except:
                                typ = 'SP'
                        else:
                            typ = node_type
                    new_label = typ + func
                t[pos].set_label(new_label)

#------------------------------------------------------------------------------------------

def build_tree(node):
    
    ''' Given a BHSA node, calls tree_string to build a tree from all lower distributional nodes then rearranges and cleans up. '''
    # 1. Build tree using atoms
    t = Tree.fromstring(tree_string(node))
    
    # 2. Move distributional nodes under function
    move_func(t)
    
    # 3. Deal with mother/daughter relationships
    # need to think about vocatives and resumption, but leave out for now
    relations = ['adj','atr','dem','mod','rec','Adju','Appo','Attr','Cmpl','Objc','PrAd','PreC','RgRc','Spec','Subj']
    move_daughter(t, relations)
    
    # 4. Deal with parallel and coordinated relations
    move_para(t)   
    
    # 5. Move linking conjunction under new para phrase
    relations = ['Link']
    move_daughter(t, relations) 
            
    # 5. Trim off empty nodes
    trim_nodes(t)
    
    # 6. Collapse redundant nodes
    collapse_nodes(t)
    
    # 7. Relabel nodes with phrase functions
    relabel_tree(t)
    
    return(t)

#------------------------------------------------------------------------------------------

def reference(node):
    book = L.u(node, 'book')
    chap = L.u(node, 'chapter')
    verse = L.i(node, 'verse')
    bk = Fs("book@en").v(book[0])
    ch = F.chapter.v(chap[0])
    if len(verse) > 1:
        vs_s = F.verse.v(verse[0])
        vs_e = F.verse.v(verse[-1])
        print('{} {}:{}-{}'.format(bk, ch, vs_s, vs_e))
    else:
        vs = F.verse.v(verse[0])
        print('{} {}:{}'.format(bk, ch, vs))

#------------------------------------------------------------------------------------------

def print_tree(t, node):
    tree = t.copy()
    slots = E.oslots.s(node)
    words = []
    for slot in slots:
        words.append(F.phono.v(slot))
                        
    for pos in tree.treepositions():
        try:
            tree[pos].label()
        except:
            tree[pos] = slots.index(int(tree[pos]))
            
    tree.pretty_print(sentence=words, unicodelines=True)

#### Step 1. Get sentences from Genesis and the Psalms

I will use a search to return lists of the sentence nodes in the books of Genesis and Psalms.

In [173]:
query1 = '''
book book=Genesis
    sentence
'''

query2 = '''
book book=Psalmi
    sentence
'''

genesis = A.search(query1)
psalms = A.search(query2)

  0.07s 4617 results
  0.07s 5104 results


#### Step 2. Get grammatical features in each sentence

Now for each sentence, we need to create a list of the grammatical features present. This will be saved as an observation in the list X, while the varaible Y stores the binary classification of the sentence(1 = Genesis, 0 = Psalms). The basic idea is to use the NLTK productions() to retrieve the grammar of the sentence. I will then use a custom function to add relevant grammatical features to the observation. I have already done a little work to transform some of the raw grammatical tags to more meaningful features, but this is one area that can use a little more work.

In [174]:
def get_features(node):
    '''Function to retrieve list of grammatical features present in a sentence node '''
    features=[]
    t = build_tree(node)
    for p in t.productions():
        label = str(p.lhs())
        if (label.startswith('NP') or
             label.startswith('PP') or
             label.startswith('VP') or
             label.startswith('REL')):
            features.append(label)
        elif label.startswith('CLAUSE'):
            if '0' in label:
                features.append('0')
            if 'Way' in label:
                features.append('Wayyiqtol')
            elif 'WQt' in label:
                features.append('Weqatal')
            elif 'W' in label:
                features.append('Syndetic')
            elif 'Z' in label:
                features.append('Zero')
            else:
                features.append(label)
        elif label == 'def':
            word = int(p.rhs()[0])
            if F.g_cons.v(word) == 'H':
                features.append('DefArt')
    return features
        

In [175]:
Y = []
X = []
nodes = []
for sent in genesis:
    node = sent[1]
    nodes.append(node)
    features = get_features(node)
    Y.append(1)
    X.append(features)

In [176]:
for sent in psalms:
    node = sent[1]
    nodes.append(node)
    features = get_features(node)
    Y.append(0)
    X.append(features)

#### Step 3. Convert feature lists to sparse array

The classifier cannot understand text strings directly, so I will use the sklearn CountVectorizer to transform the data into an array with "one hot" encoding. In short, each feature will be indexed to a column of the resulting array X_count, with the value recording the count of that feature in the sentence.

In [177]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import numpy as np

In [178]:
# This is a dummy function to bypass tokenizing since the features are already in list form
def dummy(tokens):
    return tokens

count_vect = CountVectorizer(tokenizer = dummy,
                            preprocessor = None,
                            lowercase=False)

X_counts = count_vect.fit_transform(X)
X_counts.shape

(9721, 90)

#### Step 4. Variable Selection

The resulting array has columns for 90 features. Since we are interested in interpretability, we do not want to keep this many features in the model. I will use forward selection to find the twenty most important features. (Note that this method is time consuming and won't scale well if we add more data).

In [179]:
clf = LogisticRegression(random_state=0)
sfs_forward = SequentialFeatureSelector(clf, n_features_to_select=20, direction='forward').fit(X_counts, Y)

The feature selector provides a get_support() method to return a mask with value True for the columns that were selected and False for those that were not. We can use this in combination with the get_feature_names() method from the count vectorizer above to retreive the names of the selected features. (Note that the names must be converted to an np.array to index them with the mask).

In [180]:
names = np.array(count_vect.get_feature_names())
print(names[sfs_forward.get_support()])

['CLAUSE_Adju' 'CLAUSE_Coor' 'CLAUSE_Ellp' 'CLAUSE_Objc' 'CLAUSE_xQt0'
 'CLAUSE_xQtX' 'DefArt' 'NP_Appo' 'NP_Exst' 'NP_Objc' 'NP_Time' 'NP_Voct'
 'NP_adj' 'NP_par' 'PP_Objc' 'Syndetic' 'VP_rec' 'Wayyiqtol' 'Weqatal'
 'Zero']


#### Step 5. Model fitting and Assessment

Before fitting the model, we use the transform() method to drop the columns for features that were not selected from the X_counts array.

In [181]:
X_reduced = sfs_forward.transform(X_counts)
fit = clf.fit(X_reduced, Y)

In order to assess the model we can use precicision and recall scores.

In [182]:
Y_hat = fit.predict(X_reduced)
print(metrics.classification_report(Y, Y_hat))

              precision    recall  f1-score   support

           0       0.76      0.89      0.82      5104
           1       0.84      0.68      0.76      4617

    accuracy                           0.79      9721
   macro avg       0.80      0.78      0.79      9721
weighted avg       0.80      0.79      0.79      9721



The model was able to correctly classify roughly 80% of the sentences, which is frankly not all that great. This is due in part to the fact that a sentence is a relatively small unit, though, and we would likely do much better if we attempted to classify larger chunks of text. 

We can also look at our coefficients to get a sense of the importance of each variable. In short, the size of the coefficient is related to the odds that the observation is in the book of Genesis (positive values) or not (negative values). 

In [183]:
values = fit.coef_.tolist()[0]
for name, value in sorted(zip(names[sfs_forward.get_support()],values), key=lambda x: x[1], reverse=True):
    print(name, value)

Wayyiqtol 2.5637751829412956
Weqatal 1.7922871383232206
NP_adj 1.6227008787816182
PP_Objc 1.3044821616673163
DefArt 1.2984609970510494
NP_Exst 1.1636461847131085
NP_Appo 0.9801416636840329
CLAUSE_Objc 0.7233781493720222
NP_par 0.6135745046900457
CLAUSE_Adju 0.538748376274794
Syndetic 0.42561222799574094
CLAUSE_xQt0 0.38177504822438474
CLAUSE_xQtX 0.34743457455790894
NP_Objc -0.6107632416909925
Zero -0.6413421310599826
NP_Time -0.6563686114501577
CLAUSE_Ellp -1.1038386686235526
VP_rec -1.3446500858735173
NP_Voct -1.4442116939035992
CLAUSE_Coor -1.7491930642403388


Most of these are not surprising. The presence of Wayyiqtol or Weqatal is the most significant indication that a sentence is likely from Genesis rather than the Psalms. The presence of the object marker (PP_Objc) or the consonantal article (DefArt) are also strong indicators of Genesis, while the absence of the object marker (NP_Objc) is an indicator of Psalms. Ellipsis and vocatives also make sense as indicators that a sentence is likely from the Psalms. The relative word *)asher* did not make the list despite it usually being clustered with the object marker and definite article as the "prose particles". Instead  but instead it seems that VP_rec 

The presence of a nominal phrase in an adjective position is a little surprising as an indication of Genesis, but this may relate to the "terseness" of poetry in the Psalms (see also NP_Appo). Note also that My sense is that it is simply not found frequently enough to be significant for this model where each observation is a sentence. If we looked at full chapters perhaps. 

Coordinated clauses are also prevalent in the tagging of the Psalms, but I am not sure to what extent this is influenced by expectations about genre. Namely, the coordinated clauses seem to be used to indicate poetic parallelism and I am not sure how consistent this is tagged. 